## Logistic Regression with L2 Regularization

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [3]:
# Change to binary class
y = (y > 0).astype(int)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [4]:
class LogReg:
    """
    This implementation of Logistic Regression uses mini-batch gradient descent with l2-regularization.
    """
    def __init__(self, 
                 epochs=100, 
                 tolerance = 1e-10, 
                 alpha=0.001, 
                 lambd=0, 
                 threshold=0.5, 
                 verbose=False,
                 minibatch_size=30,
                ):
        self.epochs = epochs
        self.alpha = alpha # Learning rate
        self.lambd = lambd # Regularization parameter
        self.tolerance = tolerance
        self.threshold = threshold
        self.verbose = verbose
        self.minibatch_size = minibatch_size
        
    def add_ones(self, X):
        return np.concatenate((np.ones((len(X),1)), X), axis = 1)
      
    def sigmoid(self, X, theta):
        return 1/(1 + np.exp(X@theta))
    
    def cost(self, X, y_true):
        m = X.shape[0]
        y_hat = self.sigmoid(X, self.theta)
        temp_theta = self.theta[:, 1:].copy()
        
        Cost = np.sum(-1*y_true*np.log(y_hat)-(1-y_true)*np.log(1-y_hat)) + self.lambd * np.sum(temp_theta**2)
        
        return Cost
    
    def get_minibatch(self, X, y,  minibatch):
        X_mb = X[minibatch*self.minibatch_size: (minibatch+1)*self.minibatch_size]
        y_mb = y[minibatch*self.minibatch_size: (minibatch+1)*self.minibatch_size]
        return X_mb, y_mb
    
    def fit(self, X, y):
        X = X.copy()
        X = self.add_ones(X)
        
        n, d = X.shape
        
        self.classes = np.unique(y)
        self.no_classes = len(self.classes)
        
        # Turn y into one-hot-labels if number of classes is greater than 2
        if self.no_classes > 2:
            y_encode = np.zeros((n, self.no_classes))
            y_encode[range(n), y] = 1 #numpy advanced indexing
            y = y_encode
        else:
            y = y.reshape(-1, 1)        
        
        if self.no_classes > 2:
            self.theta = np.zeros((d, self.no_classes))
        else:
            self.theta = np.zeros((d, 1))
        
        current_epoch = 1
        norm = 1
        
        no_of_minibatch = int(n/self.minibatch_size)
        
        while (norm >= self.tolerance and current_epoch < self.epochs):
            # Shuffle X for minibatch gradient descent
            shuffled_index = np.random.permutation(n)
            X_shuffled = X[shuffled_index]
            y_shuffled = y[shuffled_index]
            
            old_theta = self.theta.copy()
            theta_wo_bias = self.theta[:, 1:].copy()
            
            for mb in range(no_of_minibatch):
                X_mb, y_mb = self.get_minibatch(X, y, mb)
                
                grad = X_mb.T@(y_mb - self.sigmoid(X_mb, self.theta)) + self.lambd * np.sum(theta_wo_bias)
                
                if self.no_classes <= 2:
                    grad= grad.reshape(-1, 1)
            
                self.theta = self.theta - self.alpha*grad
            
            if self.verbose and (current_epoch%100 == 0):
                print(f'cost for {current_epoch} epoch : {self.cost(X, y)}')
            norm = np.linalg.norm(old_theta - self.theta)
            current_epoch += 1
            
        return self.theta
    
    def evaluate(self, X, y):
        """
        Returns mse loss for a dataset evaluated on the hypothesis
        """
        X = self.add_ones(X)
        return self.cost(X, y)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        if self.no_classes > 2:
            # Multiclass classification
            y_hat = np.argmax(proba, axis=1)
        elif self.no_classes == 2:
            # Binary classification
            y_hat = (proba >= self.threshold).astype(int)
        return y_hat
        
    def predict_proba(self, X):
        """
        Returns probability of predictions.
        """
        X = self.add_ones(X)
        
        return self.sigmoid(X, self.theta)

In [5]:
logreg = LogReg(verbose=True)

In [6]:
logreg.fit(X, y)

array([[ 0.200352  ],
       [ 0.30345859],
       [ 1.08722447],
       [-1.71817827],
       [-0.76690365]])

In [7]:
predictions = logreg.predict(X)
predictions = predictions.squeeze()

In [8]:
np.sum(y == predictions) / len(y)

1.0

In [9]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])